In [1]:
%%capture
!wget https://raw.githubusercontent.com/shitkov/redactor/main/yo.dat

!pip install pymorphy2
!pip install pyaspeller

In [2]:
data_path = '/content/yo.dat'

In [3]:
from redactor import redact_text

In [4]:
text = 'комиссия надбавка зависит от того, где и сколько паев открытого фонда "Рога и копыта" вы покупаете'

In [5]:
redact_text(text, '/content/yo.dat')

'Комиссия (надбавка) зависит от того, где и сколько паёв открытого фонда «Рога и копыта» вы покупаете.'